<strong>1. Load mô hình và các thư viện cần dùng</strong>

In [1]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model("model_0.9966.h5")

NGRAM = 5
MAXLEN = 39

# Các ký tự có thể xuất hiện trong encoder
alphabet = ['\x00', ' '] + list('0123456789aàáảãạăằắẳẵặâầấẩẫậeèéẻẽẹêềếểễệiìíỉĩịoòóỏõọôồốổỗộơờớởỡợuùúủũụưừứửữựyỳýỷỹỵAÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬEÈÉẺẼẸÊỀẾỂỄỆIÌÍỈĨỊOÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢUÙÚỦŨỤƯỪỨỬỮỰYỲÝỶỸỴ') + list('bcdfghjklmnpqrstvwxzBCDFGHJKLMNPQRSTVWXZdĐ')

<strong>2. Hàm encoder và decoder tương ứng với mô hình đã huấn luyện</strong>

In [2]:
# Đệm '\x00' vào cuối của các cụm ngrams có độ dài < độ dài tối đa là 39
def _encoder_data(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    for i, c in enumerate(text[:MAXLEN]):
        x[i, alphabet.index(c)] = 1
    if i < MAXLEN - 1 :
        for j in range(i + 1, MAXLEN):
            x[j, 0] = 1
    return x
def _decoder_data(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [3]:
print(_encoder_data('Tôi tên là Nguyễn Thế Anh').shape)
print(_decoder_data(_encoder_data('Tôi tên là Nguyễn Thế Anh')))

(39, 198)
Tôi tên là Nguyễn Thế Anh              


<strong>3. Các hàm sẽ sử dụng để sửa lỗi chính tả trong văn bản Tiếng Việt</strong>

- Hàm tác câu thành các ngrams

In [4]:
def _nltk_ngrams(sentence, n, maxlen):
    list_ngrams = []
    list_words = sentence.split()
    num_words = len(list_words)

    if (num_words >= n):
        for ngram in nltk.ngrams(list_words, n):
            if len(' '.join(ngram)) <= maxlen:
                list_ngrams.append(ngram)
    else:
        list_ngrams.append(tuple(list_words))

    return list_ngrams

In [5]:
_nltk_ngrams('Xuwr ný ngoon ngữ tuw nhêin', NGRAM, MAXLEN)

[('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuw'), ('ný', 'ngoon', 'ngữ', 'tuw', 'nhêin')]

- Hàm dự đoán ngram bằng mô hình

In [6]:
def _guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([_encoder_data(text)]))
    return _decoder_data(preds[0]).strip('\x00')

In [11]:
_guess(('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuwj'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


'Xử lý ngôn ngữ tự'

- Hàm thêm dấu câu khi sửa lỗi chính tả

In [21]:
def _add_punctuation(text, corrected_text):

    # Lưu dấu câu theo chỉ số của từ
    list_punctuation = {}

    # Thu thập dấu câu cho mỗi từ trong `text`
    for i, word in enumerate(text.split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            # Kiểm tra dấu câu ở đầu từ
            start_punc = ''
            for c in word:
                if c in alphabet:
                    break
                start_punc += c

            # Kiểm tra dấu câu ở cuối từ
            end_punc = ''
            for c in word[::-1]:
                if c in alphabet:
                    break
                end_punc += c
            end_punc = end_punc[::-1]

            # Lưu dấu câu vào từ điển
            list_punctuation[i] = [start_punc, end_punc]

    # Thêm dấu câu vào `corrected_text`
    result = ''
    for i, word in enumerate(corrected_text.split()):
        if i in list_punctuation:
            result += (list_punctuation[i][0] + word + list_punctuation[i][1]) + ' '
        else:
            result += word + ' '

    return result.strip()


- Hàm sửa lỗi chính tả trong câu

In [22]:
import re
from collections import Counter

def _correct(text):
    # Xóa ký tự đặc biệt
    new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
    
    ngrams = list(_nltk_ngrams(new_text, NGRAM, MAXLEN))
    guessed_ngrams = [_guess(ngram) for ngram in ngrams]
    candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
    
    for nid, ngram in enumerate(guessed_ngrams):
        for wid, word in enumerate(re.split(r'\s', ngram)):
            candidates[nid + wid].update([word])
    
    corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates)
    return _add_punctuation(text, corrected_text)


In [28]:
text = 'A aasy nói rằng: "Chong thowfi đại soos hoas hiện lay, văn bản đánh máy ddax daafn thay thế vawn barn viết tay bởi sự thuận tieejn của nó. Kefm theo đó"'
_correct(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━

'Anh yy nói rằng: "Trong thời đại số hoá hiện nay, văn bản đánh máy dãã dàn thay thế văn bản viết tay bởi sự thuận tiện của nó. Kèm theo đóó"'

<strong>4. Kết quả khi áp dụng mô hình</strong>

In [29]:
text = input()

#Văn bản sau khi sửa lỗi chính tả
result = _correct(text)
print(result)

# Xóa bỏ ký tự đặc biệt
text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
list_text = text.split()

result = re.sub(r'[^' + ''.join(alphabet) + ']', '', result)
list_result = result.split()

#Hiển thị những từ đã sửa
corrected_word =[(list_text[i], list_result[i]) for i in range(len(list_text)) if list_text[i] !=list_result[i]]
corrected_word

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━

[('A', 'Anh'),
 ('aasy', 'yy'),
 ('Chong', 'Trong'),
 ('thowfi', 'thời'),
 ('soos', 'số'),
 ('hoas', 'hoá'),
 ('lay', 'nay'),
 ('ddax', 'dãã'),
 ('daafn', 'dàn'),
 ('vawn', 'văn'),
 ('barn', 'bản'),
 ('tieejn', 'tiện'),
 ('Kefm', 'Kèm'),
 ('ó', 'óó')]